# 第10章：PyTorch与TensorFlow入门

> **学习目标**
> - 理解为什么需要深度学习框架
> - 掌握PyTorch的核心概念（张量、自动求导、模型定义）
> - 掌握TensorFlow/Keras的核心概念
> - 用两个框架实现完整的神经网络训练
> - 了解两个框架的对比和选择策略

---

## 🎯 为什么需要深度学习框架？

### 从零实现 vs 使用框架

**前9章我们做了什么？**
- 从零实现：感知机、MLP、反向传播、优化器、正则化...
- 目的：**深入理解底层原理**

**为什么要学框架？**
- ✅ **自动求导**：不用手写反向传播公式
- ✅ **GPU加速**：自动利用GPU并行计算
- ✅ **模型库**：预训练模型、标准层
- ✅ **调试工具**：可视化、监控
- ✅ **生产部署**：模型保存、推理优化

---

### 🔥 两大主流框架

| 特性 | PyTorch | TensorFlow/Keras |
|------|---------|------------------|
| **开发者** | Meta (Facebook) | Google |
| **发布时间** | 2016 | 2015 (TF) / 2015 (Keras) |
| **编程风格** | 动态图（灵活） | 静态图（TF 1.x）+ 动态图（TF 2.x） |
| **学习曲线** | 适中，更Pythonic | Keras易学，TF底层复杂 |
| **研究社区** | ⭐⭐⭐⭐⭐ 学术界主流 | ⭐⭐⭐⭐ 工业界广泛 |
| **生产部署** | TorchScript、ONNX | TensorFlow Serving、TFLite |
| **调试** | 易于调试（Python debugger） | TF 2.x改进明显 |

**本章策略：**
- 重点学习 **PyTorch**（研究和教学的首选）
- 简要介绍 **TensorFlow/Keras**（了解工业界工具）
- 同一任务用两个框架实现，便于对比

---

## ⚙️ 环境准备

In [ ]:
# 检查是否安装了PyTorch和TensorFlow
import sys

try:
    import torch
    print(f"✅ PyTorch版本: {torch.__version__}")
    print(f"   CUDA可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   GPU设备: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("❌ PyTorch未安装")
    print("   安装命令: pip install torch torchvision")

print()

try:
    import tensorflow as tf
    print(f"✅ TensorFlow版本: {tf.__version__}")
    print(f"   GPU可用: {len(tf.config.list_physical_devices('GPU')) > 0}")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"   GPU设备: {gpus[0].name}")
except ImportError:
    print("❌ TensorFlow未安装")
    print("   安装命令: pip install tensorflow")

# 通用库
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

# 设置绘图样式
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("\n✅ 环境准备完成")

---

## 1️⃣ PyTorch基础

### 📦 核心概念

PyTorch的核心是 **Tensor（张量）** 和 **Autograd（自动求导）**

#### 什么是Tensor？
- Tensor = 多维数组（类似NumPy的ndarray）
- 但可以在GPU上运行，并支持自动求导

**维度对应：**
- 0维：标量（scalar）
- 1维：向量（vector）
- 2维：矩阵（matrix）
- 3维+：张量（tensor）

---

### 💻 张量基础操作

In [ ]:
import torch

print("="*60)
print("  PyTorch张量基础")
print("="*60)

# ===== 1. 创建张量 =====
print("\n1️⃣ 创建张量的多种方式:")

# 从Python列表创建
x1 = torch.tensor([1, 2, 3, 4])
print(f"\n从列表创建: {x1}")
print(f"  形状: {x1.shape}, 数据类型: {x1.dtype}")

# 创建全零张量
x2 = torch.zeros(2, 3)  # 2x3矩阵
print(f"\n全零张量:\n{x2}")

# 创建全一张量
x3 = torch.ones(3, 4)
print(f"\n全一张量:\n{x3}")

# 创建随机张量（正态分布）
x4 = torch.randn(2, 3)  # 标准正态分布
print(f"\n随机张量（N(0,1)）:\n{x4}")

# 从NumPy数组创建
np_array = np.array([[1, 2], [3, 4]])
x5 = torch.from_numpy(np_array)
print(f"\n从NumPy创建:\n{x5}")

# ===== 2. 张量运算 =====
print("\n\n2️⃣ 张量运算:")

a = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
b = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

# 加法
print(f"\na + b =\n{a + b}")

# 逐元素乘法
print(f"\na * b (逐元素) =\n{a * b}")

# 矩阵乘法
print(f"\na @ b (矩阵乘法) =\n{a @ b}")
# 或者使用 torch.matmul(a, b)

# ===== 3. 形状操作 =====
print("\n\n3️⃣ 形状操作:")

x = torch.randn(2, 3, 4)  # 3维张量
print(f"\n原始形状: {x.shape}")

# reshape（改变形状）
x_reshaped = x.reshape(2, 12)  # 变成 2x12
print(f"reshape后: {x_reshaped.shape}")

# view（类似reshape，但要求内存连续）
x_viewed = x.view(6, 4)
print(f"view后: {x_viewed.shape}")

# transpose（转置）
x_t = x_viewed.T  # 或 x_viewed.transpose(0, 1)
print(f"转置后: {x_t.shape}")

# ===== 4. 索引和切片 =====
print("\n\n4️⃣ 索引和切片:")

x = torch.tensor([[1, 2, 3], 
                  [4, 5, 6], 
                  [7, 8, 9]])
print(f"\n原始张量:\n{x}")
print(f"第1行: {x[0, :]}")
print(f"第2列: {x[:, 1]}")
print(f"右下角2x2:\n{x[1:, 1:]}")

print("\n" + "="*60)

### 🎓 自动求导（Autograd）

**这是PyTorch的核心魔法！**

#### 原理
- 每个张量都有一个 `requires_grad` 属性
- 设置为 `True` 后，PyTorch会记录所有操作（构建计算图）
- 调用 `.backward()` 时，自动计算梯度

#### 为什么重要？
- 不用手写反向传播！
- 无论网络多复杂，自动计算所有梯度

---

In [ ]:
print("="*60)
print("  PyTorch自动求导示例")
print("="*60)

# ===== 示例1：简单函数的梯度 =====
print("\n示例1: 计算 f(x) = x^2 在 x=3 处的梯度")
print("      理论值: df/dx = 2x = 2*3 = 6")

# 创建需要梯度的张量
x = torch.tensor(3.0, requires_grad=True)
print(f"\nx = {x}, requires_grad = {x.requires_grad}")

# 前向计算
y = x ** 2
print(f"y = x^2 = {y}")

# 反向传播（计算梯度）
y.backward()  # 自动计算 dy/dx

# 查看梯度
print(f"dy/dx = {x.grad}")
print(f"✅ 梯度计算正确！")

# ===== 示例2：多变量函数 =====
print("\n\n示例2: 计算 f(x, y) = x^2 + 2xy + y^2 的梯度")
print("      理论值: ∂f/∂x = 2x + 2y, ∂f/∂y = 2x + 2y")

x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# 前向计算
f = x**2 + 2*x*y + y**2
print(f"\nf(2, 3) = {f}")

# 反向传播
f.backward()

print(f"∂f/∂x = {x.grad} (理论值: 2*2 + 2*3 = 10)")
print(f"∂f/∂y = {y.grad} (理论值: 2*2 + 2*3 = 10)")

# ===== 示例3：模拟神经网络的前向和反向传播 =====
print("\n\n示例3: 模拟神经网络 y = W @ x + b")

# 输入
x = torch.tensor([1.0, 2.0, 3.0])

# 参数（需要梯度）
W = torch.randn(2, 3, requires_grad=True)  # 权重矩阵
b = torch.randn(2, requires_grad=True)      # 偏置向量

# 前向传播
y = W @ x + b
loss = y.sum()  # 简单的损失函数

print(f"输出 y = {y}")
print(f"损失 loss = {loss}")

# 反向传播
loss.backward()

print(f"\n梯度已计算:")
print(f"  dL/dW 的形状: {W.grad.shape}")
print(f"  dL/db 的形状: {b.grad.shape}")
print(f"\n✅ PyTorch自动计算了所有梯度！")

print("\n" + "="*60)

---

## 3️⃣ PyTorch vs TensorFlow/Keras：详细对比

### 📊 代码风格对比

| 任务 | PyTorch | TensorFlow/Keras |
|------|---------|------------------|
| **模型定义** | `class MyModel(nn.Module)` | `keras.Sequential([...])` |
| **前向传播** | 显式定义 `forward()` | 自动处理 |
| **损失计算** | 手动计算 | `model.compile(loss=...)` |
| **训练循环** | 手动编写（灵活）| `model.fit()` 一行搞定 |
| **梯度更新** | 手动调用 `.backward()` | 自动处理 |

---

### 🎯 如何选择框架？

#### 选择PyTorch的场景：

1. **学术研究** ⭐⭐⭐⭐⭐
   - 需要自定义损失函数、网络层
   - 实验新算法
   - 大多数顶会论文使用PyTorch

2. **需要灵活性**
   - 动态计算图
   - 复杂的控制流（if/else/for）
   - 易于调试

3. **学习深度学习原理**
   - 代码更接近数学公式
   - 手动控制训练流程

#### 选择TensorFlow/Keras的场景：

1. **生产部署** ⭐⭐⭐⭐⭐
   - TensorFlow Serving（云端）
   - TensorFlow Lite（移动端）
   - TensorFlow.js（浏览器）

2. **快速原型**
   - Keras API极简
   - 适合工程师快速实现

3. **企业级应用**
   - Google生态系统
   - 完善的部署工具链

---

### 💡 最佳实践建议

#### 🎓 对于学习者

**推荐路径：**
1. **深入学PyTorch**
   - 理解每个步骤（前向、反向、优化）
   - 容易理解底层原理
   
2. **快速上手Keras**
   - 了解高层API的便利
   - 对比理解两种风格

#### 🏢 对于工程师

- **研究/实验阶段**：PyTorch（灵活）
- **生产部署阶段**：TensorFlow（成熟工具）
- **或者**：PyTorch训练 + ONNX转换 + TensorFlow部署

---

## 📝 综合总结

### 🎯 核心概念回顾

| 概念 | PyTorch | TensorFlow/Keras |
|------|---------|------------------|
| **张量操作** | `torch.Tensor` | `tf.Tensor` |
| **自动求导** | `requires_grad=True` + `.backward()` | `tf.GradientTape()` |
| **模型定义** | 继承 `nn.Module` | `keras.Sequential` 或 Functional API |
| **层定义** | `nn.Linear`, `nn.Conv2d` | `layers.Dense`, `layers.Conv2D` |
| **激活函数** | `F.relu`, `F.softmax` | `activation='relu'` |
| **损失函数** | `nn.CrossEntropyLoss()` | `loss='categorical_crossentropy'` |
| **优化器** | `optim.Adam()` | `optimizer='adam'` |
| **训练循环** | 手动编写 | `model.fit()` |

---

### 🚀 从零实现 vs 框架实现对比

**前9章我们做的事（从零实现）：**

```python
# 手写前向传播
z1 = X @ W1 + b1
a1 = relu(z1)
z2 = a1 @ W2 + b2
y_pred = softmax(z2)

# 手写反向传播
dW2 = a1.T @ (y_pred - y_true)
db2 = np.sum(y_pred - y_true, axis=0)
da1 = (y_pred - y_true) @ W2.T
dz1 = da1 * relu_derivative(z1)
dW1 = X.T @ dz1
db1 = np.sum(dz1, axis=0)

# 手写参数更新
W1 -= learning_rate * dW1
W2 -= learning_rate * dW2
```

**框架实现（PyTorch）：**

```python
# 前向传播
output = model(X)
loss = criterion(output, y_true)

# 反向传播+参数更新（自动！）
optimizer.zero_grad()
loss.backward()
optimizer.step()
```

**收获：**
- ✅ 理解底层原理（手写实现）
- ✅ 提高开发效率（使用框架）

---

### 📚 下一步学习建议

#### 1️⃣ 深入PyTorch

- **官方教程**: [pytorch.org/tutorials](https://pytorch.org/tutorials)
- **进阶话题**:
  - 自定义层和损失函数
  - 数据加载和增强（`torch.utils.data`）
  - 多GPU训练（`DataParallel`）
  - 模型保存和加载

#### 2️⃣ 实战项目

- **计算机视觉**: 图像分类、目标检测
- **自然语言处理**: 文本分类、情感分析
- **时间序列**: 股票预测、天气预报

#### 3️⃣ 部署技能

- **模型压缩**: 剪枝、量化
- **推理优化**: ONNX、TorchScript
- **云端部署**: AWS、GCP、Azure

---

## 🏋️ 练习题

### ⭐⭐ 练习1：实现自定义激活函数

**任务：** 在PyTorch中实现Swish激活函数：$f(x) = x \cdot \sigma(x)$

**要求：**
- 继承 `nn.Module`
- 实现 `forward()` 方法
- 在模型中使用

**提示：**
```python
class Swish(nn.Module):
    def forward(self, x):
        # 实现 x * sigmoid(x)
        pass
```

---

### ⭐⭐⭐ 练习2：添加Dropout和Batch Normalization

**任务：** 修改SimpleNN模型，添加：
1. Dropout层（p=0.5）
2. Batch Normalization层

**要求：**
- 在两层之间添加
- 对比有无这些层的效果

---

### ⭐⭐⭐⭐ 练习3：实现学习率调度

**任务：** 使用第9章学到的学习率调度策略。

**要求：**
1. 使用 `torch.optim.lr_scheduler.CosineAnnealingLR`
2. 可视化学习率变化
3. 对比固定学习率和调度学习率的效果

**参考：**
```python
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
for epoch in range(num_epochs):
    train(...)
    scheduler.step()  # 更新学习率
```

---

### ⭐⭐⭐⭐⭐ 练习4：实现完整的MNIST训练（真实数据）

**任务：** 使用真正的MNIST数据集（28x28图像）。

**要求：**
1. 使用 `torchvision.datasets.MNIST` 加载数据
2. 定义卷积神经网络（CNN）结构
3. 训练并达到 >98% 准确率
4. 可视化错误预测的样本

**提示：**
```python
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
```

---

## 🎓 学习检查点

完成本章后，你应该能够：

- [ ] **理解为什么需要深度学习框架** 而不是一直从零实现
- [ ] **掌握PyTorch核心概念** 张量、自动求导、模型定义
- [ ] **编写完整的PyTorch训练循环** 前向→损失→反向→优化
- [ ] **了解TensorFlow/Keras的使用** Sequential API和model.fit()
- [ ] **对比两个框架的优劣** 并根据场景选择
- [ ] **将前9章知识应用到框架中** 理解框架背后的原理

---

## 🎉 恭喜！神经网络模块完成！

**你已经完成了从零到一的完整旅程：**

1. ✅ **理论基础**（第1-2章）：感知机、单层网络
2. ✅ **核心算法**（第3-4章）：MLP、反向传播
3. ✅ **优化技术**（第5-7章）：损失函数、优化器、正则化
4. ✅ **高级技巧**（第8-9章）：权重初始化、训练诊断
5. ✅ **工业工具**（第10章）：PyTorch/TensorFlow

---

### 🚀 下一步：应用与深入

#### 继续本教程：

1. **计算机视觉**（CNN）
   - 卷积层、池化层
   - 经典架构（LeNet、VGG、ResNet）

2. **序列模型**（RNN/Transformer）
   - 循环神经网络
   - 注意力机制

3. **生成模型**
   - 变分自编码器（VAE）
   - 生成对抗网络（GAN）

#### 独立项目：

- 参加Kaggle竞赛
- 复现经典论文
- 构建自己的深度学习应用

---

**记住：**
> **深度学习 = 理论 + 实践 + 调参经验**
>
> 前9章给了你**理论和实现能力**
>
> 第10章给了你**工程工具**
>
> 现在需要的是**大量实践**！

**Happy Deep Learning! 🎯**

In [ ]:
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers

    print("="*60)
    print("  TensorFlow/Keras完整训练示例")
    print("="*60)

    # 使用相同的数据
    print("\n1️⃣ 使用相同的训练数据...")
    print(f"训练集大小: {len(X_train)}")
    print(f"测试集大小: {len(X_test)}")

    # ===== 方式1：Sequential API =====
    print("\n2️⃣ 定义模型（Sequential API）...")
    
    model_keras = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(64,)),
        layers.Dense(10, activation='softmax')
    ])
    
    print(model_keras.summary())
    
    # ===== 3. 编译模型 =====
    print("\n3️⃣ 编译模型...")
    
    model_keras.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("优化器: Adam")
    print("损失函数: SparseCategoricalCrossentropy")
    
    # ===== 4. 训练模型 =====
    print("\n4️⃣ 开始训练...")
    
    history = model_keras.fit(
        X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=0  # 不打印每个epoch
    )
    
    # 每5个epoch打印一次
    for epoch in range(0, 20, 5):
        train_loss = history.history['loss'][epoch]
        val_acc = history.history['val_accuracy'][epoch]
        print(f\"Epoch [{epoch+1}/20] Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}\")
    
    print("\n✅ 训练完成！")
    final_acc = history.history['val_accuracy'][-1]
    print(f\"最终测试准确率: {final_acc:.4f}\")
    
    # ===== 5. 可视化对比 =====
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 训练损失
    axes[0].plot(history.history['loss'], linewidth=2, color='steelblue', label='TensorFlow')
    axes[0].plot(train_losses, linewidth=2, color='coral', label='PyTorch', linestyle='--')
    axes[0].set_title('训练损失对比', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 测试准确率
    axes[1].plot(history.history['val_accuracy'], linewidth=2, color='forestgreen', label='TensorFlow')
    axes[1].plot(test_accuracies, linewidth=2, color='orange', label='PyTorch', linestyle='--')
    axes[1].set_title('测试准确率对比', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('framework_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n" + "="*60)

except ImportError:
    print("❌ TensorFlow未安装，跳过Keras示例")
    print("   安装命令: pip install tensorflow")

---

## 2️⃣ TensorFlow/Keras基础

### 📦 Keras：高层API

**TensorFlow 2.x 的核心：** 将Keras作为官方高层API

**优势：**
- ✅ 极简的API设计
- ✅ 快速原型开发
- ✅ 生产部署成熟

**核心概念：**
- **Sequential API**：适合简单的顺序模型
- **Functional API**：适合复杂模型（多输入/输出、分支结构）

---

### 💻 Keras模型定义和训练

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("="*60)
print("  PyTorch完整训练示例：手写数字识别")
print("="*60)

# ===== 1. 准备数据 =====
print("\n1️⃣ 准备数据...")

# 使用sklearn的手写数字数据集（8x8图像，类似MNIST但更小）
digits = load_digits()
X, y = digits.data, digits.target

# 标准化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 转换为PyTorch张量
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"训练集大小: {len(X_train)}")
print(f"测试集大小: {len(X_test)}")
print(f"特征维度: {X_train.shape[1]}")
print(f"类别数: {len(np.unique(y))}")

# ===== 2. 定义模型 =====
print("\n2️⃣ 定义模型...")

model = SimpleNN(input_size=64, hidden_size=128, num_classes=10)
print(f"模型参数数量: {sum(p.numel() for p in model.parameters()):,}")

# ===== 3. 定义损失函数和优化器 =====
print("\n3️⃣ 定义损失函数和优化器...")

# 交叉熵损失（自动包含softmax）
criterion = nn.CrossEntropyLoss()

# Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("损失函数: CrossEntropyLoss")
print("优化器: Adam (lr=0.001)")

# ===== 4. 训练循环 =====
print("\n4️⃣ 开始训练...")

num_epochs = 20
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    # ===== 训练模式 =====
    model.train()  # 设置为训练模式
    epoch_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        # 前向传播
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # 反向传播和优化
        optimizer.zero_grad()  # 清零梯度
        loss.backward()        # 计算梯度
        optimizer.step()       # 更新参数
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    # ===== 验证模式 =====
    model.eval()  # 设置为评估模式
    with torch.no_grad():  # 不计算梯度
        test_outputs = model(X_test_tensor)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = (predicted == y_test_tensor).float().mean().item()
        test_accuracies.append(accuracy)
    
    # 打印进度
    if (epoch + 1) % 5 == 0:
        print(f\"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}, Test Acc: {accuracy:.4f}\")

print(\"\\n✅ 训练完成！\")
print(f\"最终测试准确率: {test_accuracies[-1]:.4f}\")

# ===== 5. 可视化训练过程 =====
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 训练损失
axes[0].plot(train_losses, linewidth=2, color='steelblue')
axes[0].set_title('训练损失曲线', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True, alpha=0.3)

# 测试准确率
axes[1].plot(test_accuracies, linewidth=2, color='forestgreen')
axes[1].set_title('测试准确率曲线', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.savefig('pytorch_training.png', dpi=150, bbox_inches='tight')
plt.show()

print(\"\\n\" + \"=\"*60)

### 🚀 完整的PyTorch训练循环

**标准训练流程：**
1. 定义模型
2. 定义损失函数和优化器
3. 加载数据
4. 训练循环（前向传播 → 计算损失 → 反向传播 → 更新参数）
5. 验证/测试

---

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# ===== 方式1：继承nn.Module =====
class SimpleNN(nn.Module):
    """
    简单的多层感知机（MLP）
    
    结构: 输入层(784) -> 隐藏层(128) -> 输出层(10)
    """
    
    def __init__(self, input_size=784, hidden_size=128, num_classes=10):
        """
        初始化网络层
        
        参数:
            input_size: 输入特征维度（如MNIST的28*28=784）
            hidden_size: 隐藏层神经元数
            num_classes: 输出类别数
        """
        super(SimpleNN, self).__init__()  # 调用父类初始化
        
        # 定义网络层
        self.fc1 = nn.Linear(input_size, hidden_size)  # 全连接层1
        self.fc2 = nn.Linear(hidden_size, num_classes)  # 全连接层2
        
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: 输入张量, shape (batch_size, input_size)
        
        返回:
            输出张量, shape (batch_size, num_classes)
        """
        # 第一层 + ReLU激活
        x = self.fc1(x)      # (batch_size, hidden_size)
        x = F.relu(x)        # ReLU激活函数
        
        # 第二层（不加激活，后面会用softmax）
        x = self.fc2(x)      # (batch_size, num_classes)
        
        return x


# ===== 方式2：使用Sequential（适合简单模型）=====
model_sequential = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)


# ===== 创建并查看模型 =====
print("="*60)
print("  PyTorch模型定义")
print("="*60)

# 实例化模型
model = SimpleNN(input_size=784, hidden_size=128, num_classes=10)

print("\n模型结构:")
print(model)

print("\n\n模型参数:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

# 计算总参数数量
total_params = sum(p.numel() for p in model.parameters())
print(f"\n总参数数量: {total_params:,}")

# ===== 测试前向传播 =====
print("\n\n测试前向传播:")
# 创建一个batch的假数据
batch_size = 16
fake_input = torch.randn(batch_size, 784)
print(f"输入形状: {fake_input.shape}")

# 前向传播
output = model(fake_input)
print(f"输出形状: {output.shape}")
print(f"✅ 前向传播成功！")

print("\n" + "="*60)

### 🏗️ PyTorch模型定义

在PyTorch中，定义模型有两种主要方式：

#### 方式1：继承 `nn.Module`（推荐）

这是最标准、最灵活的方式。

**核心步骤：**
1. 继承 `torch.nn.Module`
2. 在 `__init__` 中定义层
3. 在 `forward` 中定义前向传播逻辑

---